In [ ]:
## Import all the necessary packages

import random

from googlesearch import search 

import pandas as pd

import requests

from bs4 import BeautifulSoup

import re

from datetime import date



In [ ]:
## RUN EVERY DAY

In [ ]:
## Upload the data

themes_df = pd.read_csv('D:\Projects\Mind_Expander\data_themes.csv', index_col=0)
print(themes_df.head())
print(themes_df.shape)

general_history = pd.read_csv('D:\Projects\Mind_Expander\data_feedback.csv', index_col=0)
print(general_history.head())

In [ ]:
## Customization by the reader

num_subjects = int(input('How many new subject do you want to discover today? '))
num_articles = int(input('How many articles do you want to read about each subject? '))
freshness = input('What is your timeline of investigation? M for month or Y for year ')
tbs = ['qdr:y' if temp == 'Y' else 'qdr:m' for temp in freshness]

In [ ]:
## Select the category of the day


# Generate the number of subject the reader specified (in two steps)

# First, select the theme of the day (some themes have more categories)
themes_of_day = []

# Loop to generate different themes (if reader specified more than one subject)
for i in range(0,num_subjects):
    theme_int = random.randint(0,len(themes_df.themes.unique())-1)
    theme_name = themes_df.themes.unique()[theme_int]
    themes_of_day.append(theme_name)


# Second, select the category of the day 
categories_of_day = []

# Loop to generate different categories (if reader specified more than one subject)
for theme in themes_of_day:
    category_int = random.randint(0,len(themes_df[themes_df.themes == theme])-1)
    category_name = themes_df[themes_df.themes == theme].categories.iloc[category_int]
    categories_of_day.append(category_name)


## Create the query
read_of_day = []

for i in range(len(categories_of_day)):
    query_google = 'help me understand ' + str(categories_of_day[i]) + ' with an article'
  
    for article in search(query_google, tld="com", lang='en', tbs=tbs, start= 0, stop=num_articles, pause=15, tpe='nws'):
        print('For {}, please read: {}'.format(categories_of_day[i], article))    
        read_of_day.append(article)
        


In [ ]:
## Generate feedback on the articles

grades = []

for art in read_of_day:
    grade = int(input('How would you rate the article (over 10)? '))
    grades.append(grade)
    
## Organize the feedback in a dataframe

# Keep track of meta data (date, theme and category)
theme = []
category = []
dates = []

for i in range(num_subjects):
    for j in range(num_articles):
        theme.append(themes_of_day[i])
        category.append(categories_of_day[i])
        dates.append(date.today())

# Compile the dataframe
history = pd.DataFrame(list(zip(dates, read_of_day, grades,category,theme)), 
                       columns=['date','article', 'grade', 'category', 'themes'])

# Collect the category already read
to_delete = category

# Update the theme dataframe
new_themes_df = themes_df
for cat in to_delete:
    new_themes_df = new_themes_df.drop(new_themes_df[new_themes_df.categories == cat].index)
new_themes_df.to_csv('D:\Projects\Mind_Expander\data_themes.csv')

# Update the general feedback dataframe
general_history = pd.concat([general_history, history])
general_history.to_csv('D:\Projects\Mind_Expander\data_feedback.csv')
